In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
def preprocess(filename) -> pd.DataFrame:
    df = pd.read_csv(filename, nrows= 30000000 )
    df.drop(columns=['key'],inplace=True)
    df.drop(columns=['pickup_datetime'],inplace=True)
    df.drop(columns=['passenger_count'],inplace=True)

    # df.head()
    df.dropna(inplace=True)

    # df.drop(df[df['passenger_count']>=7].index, inplace=True)
    # df.drop(df[df['passenger_count']<=0].index, inplace=True)
    df.drop(df[df['fare_amount']<=0.1].index, inplace=True)
    df.drop(df[df['fare_amount']>=500].index, inplace=True)
    df.drop(df[df['pickup_latitude']<=39].index, inplace=True)
    df.drop(df[df['pickup_latitude']>=41].index, inplace=True)
    df.drop(df[df['pickup_longitude']>=-70].index, inplace=True)
    df.drop(df[df['pickup_longitude']<=-80].index, inplace=True)
    df.drop(df[df['dropoff_latitude']<=35].index, inplace=True)
    df.drop(df[df['dropoff_latitude']>=45].index, inplace=True)
    df.drop(df[df['dropoff_longitude']>=-73].index, inplace=True)
    df.drop(df[df['dropoff_longitude']<=-75].index, inplace=True)
    df['pickup_longitude'] = df['pickup_longitude'].div(2)
    df['dropoff_longitude'] = df['dropoff_longitude'].div(2)
    return df

df = preprocess('train.csv')
train = df.to_numpy(dtype=float)
np.random.shuffle(train)

trainX = train[:,1:]
trainY = train[:,0]

In [3]:
df = pd.read_csv('test.csv')
df.drop(columns=['pickup_datetime'],inplace=True)
submit = df['key'].to_numpy()
df.drop(columns=['key'],inplace=True)
df.drop(columns=['passenger_count'],inplace=True)
df['pickup_longitude'] = df['pickup_longitude'].div(2)
df['dropoff_longitude'] = df['dropoff_longitude'].div(2)
test = df.to_numpy(dtype=float)
print(test)
print(submit)
df.describe()

[[-36.98666     40.76380539 -36.99071503  40.74383545]
 [-36.99343109  40.71938324 -36.99944305  40.73920059]
 [-36.991262    40.75126    -36.989827    40.746139  ]
 ...
 [-36.99580002  40.72660828 -36.89487076  40.6470108 ]
 [-36.99278641  40.73543167 -36.96958923  40.80173111]
 [-36.99401093  40.75407028 -37.00014114  40.75922012]]
['2015-01-27 13:08:24.0000002' '2015-01-27 13:08:24.0000003'
 '2011-10-08 11:53:44.0000002' ... '2015-04-19 20:44:15.0000001'
 '2015-01-31 01:05:19.0000005' '2015-01-18 14:06:23.0000006']


,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,9914.000000,9914.000000,9914.000000,9914.000000
mean,-36.987361,40.751041,-36.986828,40.751743
std,0.021387,0.033541,0.019536,0.035435
min,-37.126097,40.573143,-37.131621,40.568973
25%,-36.996250,40.736125,-36.995623,40.735254
50%,-36.991163,40.753051,-36.990008,40.754065
75%,-36.984007,40.767113,-36.982030,40.768757
max,-36.493266,41.709555,-36.495481,41.696683


In [4]:
clf = KNeighborsRegressor(n_neighbors = 7)
clf.fit(trainX,trainY)
predict = clf.predict(test)
print(predict)

[ 9.78571429  8.95714286  4.47142857 ... 51.69571429 15.82857143
  7.8       ]


In [5]:
output = np.c_[submit.T,predict.T]
print(output)


[['2015-01-27 13:08:24.0000002' 9.785714285714286]
 ['2015-01-27 13:08:24.0000003' 8.957142857142857]
 ['2011-10-08 11:53:44.0000002' 4.471428571428571]
 ...
 ['2015-04-19 20:44:15.0000001' 51.69571428571429]
 ['2015-01-31 01:05:19.0000005' 15.828571428571431]
 ['2015-01-18 14:06:23.0000006' 7.8]]


In [6]:
df = pd.DataFrame(output)
df.to_csv('output.csv', index=None, header=['key','fare_amount'])

In [7]:
# clf = GradientBoostingRegressor(n_estimators=1000)
# clf.fit(trainX,trainY)
# predict = clf.predict(test)
# output = np.c_[submit.T,predict.T]
# print(output)

In [ ]:
# df = pd.DataFrame(output)
# df.to_csv('submission.csv', index=None, header=['key','fare_amount'])